In [1]:
!pip3 install pyvi keras gensim
!pip3 install --upgrade keras tensorflow
!pip3 install "tensorflow-text==2.13.*"

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached keras-3.0.0-py3-none-any.whl.metadata (5.3 kB)
  Using cached rich-13.7.0-py3-none-any.whl.metadata (18 kB)
  Using cached namex-0.0.7-py3-none-any.whl (5.8 kB)
  Using cached dm_tree-0.1.8-cp39-cp39-macosx_11_0_arm64.whl (110 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement tensorflow-text==2.13.* (from versions: none)
ERROR: No matching distribution found for tensorflow-text==2.13.*


In [2]:
from __future__ import division, print_function
from gensim import models
# from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
# from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import re
from string import digits
from collections import Counter
from pyvi import ViTokenizer

In [3]:
data_train = pd.read_csv("./vlsp_sentiment_train.csv", sep='\t')
data_train.columns =['Class', 'Data']
data_test = pd.read_csv("./vlsp_sentiment_test.csv", sep='\t')
data_test.columns =['Class', 'Data']

In [4]:
print(data_train.shape)
print(data_test.shape)

(5100, 2)
(1050, 2)


In [5]:
labels = data_train.iloc[:, 0].values
reviews = data_train.iloc[:, 1].values

In [6]:
with open("./stopwords.txt", "r", encoding="utf8") as f:
    stopwords = f.read().splitlines()

def remove_stop_words(text, stopwords):
    words = text.split()
    words = [word for word in words if word not in stopwords]
    text = ' '.join(words)
    return text

In [7]:
encoded_labels = []

for label in labels:
    if label == -1:
        encoded_labels.append([1,0,0])
    elif label == 0:
        encoded_labels.append([0,1,0])
    else:
        encoded_labels.append([0,0,1])

encoded_labels = np.array(encoded_labels)  

In [8]:
reviews_processed = []
unlabeled_processed = [] 
for review in reviews:
    review_cool_one = ''.join([char for char in review if char not in digits])
    # review_cool_one = remove_stop_words(review_cool_one, stopwords)
    reviews_processed.append(review_cool_one)

In [9]:
#Use PyVi for Vietnamese word tokenizer
word_reviews = []
all_words = []
for review in reviews_processed:
    review = ViTokenizer.tokenize(review.lower())
    word_reviews.append(review.split())

In [10]:
EMBEDDING_DIM = 400 # how big is each word vector
MAX_VOCAB_SIZE = 10000 # how many unique words to use (i.e num rows in embedding vector)
MAX_SEQUENCE_LENGTH = 300 # max number of words in a comment to use

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [12]:
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, lower=True, char_level=False)
tokenizer.fit_on_texts(word_reviews)
sequences_train = tokenizer.texts_to_sequences(word_reviews)
word_index = tokenizer.word_index

In [13]:
data = pad_sequences(sequences_train, maxlen=MAX_SEQUENCE_LENGTH)
labels = encoded_labels

In [14]:
print('Shape of X train and X validation tensor:',data.shape)
print('Shape of label train and validation tensor:', labels.shape)

Shape of X train and X validation tensor: (5100, 300)
Shape of label train and validation tensor: (5100, 3)


In [15]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors

word_vectors = KeyedVectors.load_word2vec_format("./vi-model-CBOW.bin", binary=True)


vocabulary_size=min(len(word_index)+1,MAX_VOCAB_SIZE)
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
for word, i in word_index.items():
    if i>=MAX_VOCAB_SIZE:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

del(word_vectors)

from keras.layers import Embedding
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=True)

In [16]:
from keras.layers import Bidirectional, LSTM
from keras import regularizers
from tensorflow.keras.optimizers import Adam

sequence_length = data.shape[1]
drop = 0.5

inputs = Input(shape=(sequence_length,))
embedding = embedding_layer(inputs)

reshape = Reshape((sequence_length,EMBEDDING_DIM))(embedding)

# Use bidirectional LSTM layers
lstm_2 = Bidirectional(LSTM(1024, return_sequences=True))(reshape)
lstm_1 = Bidirectional(LSTM(512, return_sequences=True))(lstm_2)
lstm_0 = Bidirectional(LSTM(256))(lstm_1)

dropout = Dropout(drop)(lstm_0)
output = Dense(units=3, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout)

model = Model(inputs, output)

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 300)]             0         
                                                                 
 embedding (Embedding)       (None, 300, 400)          3167600   
                                                                 
 reshape (Reshape)           (None, 300, 400)          0         
                                                                 
 bidirectional (Bidirection  (None, 300, 2048)         11673600  
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 300, 1024)         10489856  
 onal)                                                           
                                                                 
 bidirectional_2 (Bidirecti  (None, 512)               262348

In [17]:
# early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
# lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=1, factor=0.5, min_lr=0.00001)
callbacks_list = [early_stopping]

np.random.seed(42)

x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, shuffle=True, random_state=42)

model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=10, batch_size=256, callbacks=callbacks_list, shuffle=True)

Epoch 1/10
16/16 [==============================] - 1108s 71s/step - loss: 1.2589 - accuracy: 0.3772 - val_loss: 1.0696 - val_accuracy: 0.4549
Epoch 2/10
16/16 [==============================] - 1198s 76s/step - loss: 1.0411 - accuracy: 0.5032 - val_loss: 1.0159 - val_accuracy: 0.4833
Epoch 3/10
16/16 [==============================] - 1477s 94s/step - loss: 0.9128 - accuracy: 0.5961 - val_loss: 0.9439 - val_accuracy: 0.5775
Epoch 4/10
16/16 [==============================] - 1033s 64s/step - loss: 0.8039 - accuracy: 0.6642 - val_loss: 0.8731 - val_accuracy: 0.6275
Epoch 5/10
16/16 [==============================] - 950s 60s/step - loss: 0.6829 - accuracy: 0.7390 - val_loss: 0.8948 - val_accuracy: 0.6333
Epoch 6/10
16/16 [==============================] - 1049s 66s/step - loss: 0.5676 - accuracy: 0.7858 - val_loss: 0.8861 - val_accuracy: 0.6343
Epoch 7/10
16/16 [==============================] - 1054s 67s/step - loss: 0.5024 - accuracy: 0.8167 - val_loss: 0.9356 - val_accuracy: 0.6529


In [18]:
labels_test = data_test.iloc[:, 0].values
reviews_test = data_test.iloc[:, 1].values

In [19]:
encoded_labels_test = []

for label_test in labels_test:
    if label_test == -1:
        encoded_labels_test.append([1,0,0])
    elif label_test == 0:
        encoded_labels_test.append([0,1,0])
    else:
        encoded_labels_test.append([0,0,1])

encoded_labels_test = np.array(encoded_labels_test)  

In [20]:
reviews_processed_test = []
unlabeled_processed_test = [] 
for review_test in reviews_test:
    review_cool_one = ''.join([char for char in review_test if char not in digits])
    # review_cool_one = remove_stop_words(review_cool_one, stopwords)
    reviews_processed_test.append(review_cool_one)

In [21]:
#Use PyVi for Vietnamese word tokenizer
word_reviews_test = []
all_words = []
for review_test in reviews_processed_test:
    review_test = ViTokenizer.tokenize(review_test.lower())
    word_reviews_test.append(review_test.split())

In [22]:
sequences_test = tokenizer.texts_to_sequences(word_reviews_test)
data_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)
labels_test = encoded_labels_test

In [23]:
print('Shape of X train and X validation tensor:',data_test.shape)
print('Shape of label train and validation tensor:', labels_test.shape)

Shape of X train and X validation tensor: (1050, 300)
Shape of label train and validation tensor: (1050, 3)


In [24]:
score = model.evaluate(data_test, labels_test)

33/33 [==============================] - 122s 4s/step - loss: 0.8402 - accuracy: 0.6800


In [25]:
print("%s: %.2f%%" % (model.metrics_names[0], score[0]*100))
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

loss: 84.02%
accuracy: 68.00%
